In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

In [3]:
def url_to_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    return soup

In [4]:
start_lst = [1,51,151,201,251,301,351,401,451,501,551,601,651,701,751,801,851,901,951,1001,1051,1101,1151,
             1201,1251,1301,1351,1401,1451,1501,1551,1601,1651,1701]
url = 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start={}&explore=title_type,genres&ref_=adv_nxt'
imdb_links = []

for num in start_lst:
    imdb_links.append(url.format(num))

In [5]:
imdb_links

['https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=1&explore=title_type,genres&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=51&explore=title_type,genres&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=151&explore=title_type,genres&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=201&explore=title_type,genres&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=251&explore=title_type,genres&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=301&explore=title_type,genres&ref_=adv_nxt',
 'https://www.imdb.com/search/title/?title_type=movie&genres=sci-fi&sort=boxoffice_gross_us,desc&start=351&explore=

In [10]:
titles = []
ratings = []
runtimes = []

df_dict = {}
'''df_dict = {'Title':None,'Gross Revenue':None,
           'Rating':None,'Runtime':None,'Num of Top Actors':None,
           'Director':None,'Writer':None,'Budget':None,'Robots':None,
           'Battle':None,'Time travel':None,'Aliens':None,
           'Space Travel':None}'''
for i in imdb_links:
    soup = url_to_soup(i)
    for j in soup.find_all(class_='lister-item-content'):
        titles.append(j.find('a').contents[0])  
    for r in soup.find_all(class_='lister-item-content'):
        if r.find('p', {'class':'text-muted'}):
            if r.find('span',{'class': 'runtime'}):
                rt=r.find('span',{'class': 'runtime'}).text
                runtime=re.sub('[^0-9]','',rt)
                #print(runtime)
                runtimes.append(runtime)
            else:
                runtimes.append('NA')  
    for i in soup.find_all(class_='lister-item-content'):
        if i.find('p', {'class':'text-muted'}):
            if i.find('span',{'class': 'certificate'}):
                ratings.append(i.find('span',{'class': 'certificate'}).text)
            else:
                ratings.append('NA')
    
    
df_dict['Title']=titles
#df_dict['Rating']=ratings 


In [11]:
#df_dict['Runtime']=runtimes

In [12]:
len(runtimes)

1700

In [13]:
#creating list (id_lst) of IDs to access individual pages for each movie

temp_lst = []
new_lst = []
id_lst = []

for i in imdb_links:
    soup = url_to_soup(i)
    for m in soup.find_all(class_='lister-item-content'):
        temp_lst.append(m.find('a'))
    
    for q in temp_lst:
        newq = str(q)
        if newq not in new_lst:
            new_lst.append(newq)
    
    for x in new_lst:
        title_id = re.findall('(?<=href=")(.*)(?=">)',x)
        #print(title_id)
        if title_id not in id_lst:
            id_lst.append(title_id)
            
id_lst = [item for sublist in id_lst for item in sublist]


In [14]:
def generate_movie_links(url,id_lst):
    lst = []
    for link_id in id_lst:
        lst.append(url.format(link_id))
    return lst

In [15]:
mojo_url = 'https://www.boxofficemojo.com{}?ref_=bo_se_r_1'
imdb_url = 'https://www.imdb.com/{}?ref_=adv_li_tt'

In [16]:
mojo_url_lst = generate_movie_links(mojo_url, id_lst)

In [17]:
mojo_url_lst

['https://www.boxofficemojo.com/title/tt2488496/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt4154796/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt0499549/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt1825683/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt4154756/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt0369610/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt0848228/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt2527336/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt3606756/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt3748528/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt2527338/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt0120915/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt2395427/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt0083866/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt4154664/?ref_=bo_se_r_1',
 'https://

In [47]:
revenues = []

#test = ['https://www.boxofficemojo.com/title/tt2488496/?ref_=bo_se_r_1']

for i in mojo_url_lst:
    soup = url_to_soup(i)
    try:
        if soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[2]: 
            #print(soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money'))
            worldwide_rev=soup.find(class_='mojo-performance-summary-table').find_all('span', class_='money')[2].text
            worldwide_rev=re.sub('\D','',worldwide_rev)
            revenues.append(worldwide_rev)
    except:
        revenues.append(np.nan)

        

In [48]:
df_dict['Revenue']=revenues

In [49]:
df_dict['Runtime']=runtimes

In [50]:
'''revenues_int=[]
for i in revenues:
    if i != np.nan:
        new=int(i)
        revenues_int.append(new)
revenues_int'''

'revenues_int=[]\nfor i in revenues:\n    if i != np.nan:\n        new=int(i)\n        revenues_int.append(new)\nrevenues_int'

In [51]:
top_actors_links = []
pg_lst = [1,101,201,301,401,501]
url = 'https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/{}'
for num in pg_lst:
    top_actors_links.append(url.format(num))

In [52]:
top_actors_links

['https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/1',
 'https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/101',
 'https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/201',
 'https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/301',
 'https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/401',
 'https://www.the-numbers.com/box-office-star-records/worldwide/lifetime-acting/top-grossing-stars/501']

In [53]:
top_actors=[]
for link in top_actors_links:
    soup = url_to_soup(link)
    for j in soup.find_all(id='page_filling_chart'):
        for x in j.find_all(class_='data'):
            top_actors.append(x.findNext().text)

In [54]:
imdb_url_lst = generate_movie_links(imdb_url, id_lst)

In [55]:
imdb_url_lst

['https://www.imdb.com//title/tt2488496/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4154796/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0499549/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt1825683/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4154756/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0369610/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0848228/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt2527336/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt3606756/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt3748528/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt2527338/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0120915/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt2395427/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0083866/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4154664/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt1951264/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4881806/?ref_=adv_li_tt',
 'https://www.

In [56]:
actor_page_mojo = 'https://www.boxofficemojo.com{}credits/?ref_=bo_tt_tab#tabs'

In [57]:
actor_links = generate_movie_links(actor_page_mojo,id_lst)

In [58]:
actor_links

['https://www.boxofficemojo.com/title/tt2488496/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt4154796/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt0499549/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt1825683/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt4154756/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt0369610/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt0848228/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt2527336/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt3606756/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt3748528/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt2527338/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt0120915/credits/?ref_=bo_tt_tab#tabs',
 'https://www.boxofficemojo.com/title/tt2395427/cred

In [59]:
mojotest=['https://www.boxofficemojo.com/title/tt2488496/credits/?ref_=bo_tt_tab#tabs']
actors=[]
for link in actor_links:
    sublist=[]
    soup = url_to_soup(link)
    table = soup.find('table', {'id':'principalCast'})
    try:
        rows = [row for row in table.find_all('tr')]
        #name = rows[1].find_all('td')[0].find('a').text
        for row in rows[1:6]:
            items = row.find_all('td')
            link = items[0].find('a')
            name= link.text.rstrip('\n')
            #actor_lst.append(name)
            sublist.append(name)   
        actors.append(sublist) 
    except:
        sublist.append('NA')
        actors.append(sublist)
    
'''        
actor_lst=[]

for row in rows[1:6]:
    items = row.find_all('td')
    link = items[0].find('a')
    name= link.text
    actor_lst.append(name)
    
actor_lst'''
        

"        \nactor_lst=[]\n\nfor row in rows[1:6]:\n    items = row.find_all('td')\n    link = items[0].find('a')\n    name= link.text\n    actor_lst.append(name)\n    \nactor_lst"

In [60]:
actors

[['Daisy Ridley', 'John Boyega', 'Oscar Isaac', 'Domhnall Gleeson'],
 ['Robert Downey Jr.', 'Chris Evans', 'Mark Ruffalo', 'Chris Hemsworth'],
 ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Michelle Rodriguez'],
 ['Chadwick Boseman', 'Michael B. Jordan', "Lupita Nyong'o", 'Danai Gurira'],
 ['Robert Downey Jr.', 'Chris Hemsworth', 'Mark Ruffalo', 'Chris Evans'],
 ['Chris Pratt', 'Bryce Dallas Howard', 'Ty Simpkins', 'Judy Greer'],
 ['Robert Downey Jr.', 'Chris Evans', 'Scarlett Johansson', 'Jeremy Renner'],
 ['Daisy Ridley', 'John Boyega', 'Mark Hamill', 'Carrie Fisher'],
 ['Craig T. Nelson', 'Holly Hunter', 'Sarah Vowell', 'Huck Milner'],
 ['Felicity Jones', 'Diego Luna', 'Alan Tudyk', 'Donnie Yen'],
 ['Daisy Ridley', 'John Boyega', 'Oscar Isaac', 'Adam Driver'],
 ['Ewan McGregor', 'Liam Neeson', 'Natalie Portman', 'Jake Lloyd'],
 ['Robert Downey Jr.', 'Chris Evans', 'Mark Ruffalo', 'Chris Hemsworth'],
 ['Henry Thomas', 'Drew Barrymore', 'Peter Coyote', 'Dee Wallace'],
 ['Bri

In [61]:
len(actors)

1700

In [62]:
#rank actors
actor_rankings=[]
for sublist in actors:
    count=0
    for actor in sublist:  
        if actor in top_actors:
            count+=1
    actor_rankings.append(count)
len(actor_rankings)
            

1700

In [63]:
actor_rankings

[4,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 2,
 1,
 4,
 3,
 3,
 1,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 2,
 3,
 4,
 2,
 3,
 2,
 3,
 4,
 4,
 1,
 2,
 3,
 3,
 2,
 4,
 2,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 2,
 1,
 4,
 2,
 1,
 3,
 2,
 3,
 4,
 3,
 4,
 4,
 3,
 2,
 3,
 3,
 2,
 2,
 1,
 1,
 2,
 4,
 2,
 1,
 0,
 2,
 1,
 3,
 1,
 3,
 2,
 3,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 1,
 3,
 3,
 4,
 4,
 2,
 3,
 1,
 0,
 2,
 0,
 2,
 2,
 3,
 2,
 2,
 1,
 3,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 3,
 3,
 3,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 1,
 2,
 4,
 4,
 1,
 1,
 3,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 0,
 3,
 3,
 0,
 1,
 0,
 0,
 0,
 1,
 3,
 4,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 2,
 2,
 0,
 3,
 2,
 1,
 0,
 2,
 4,
 1,
 1,
 0,
 0,
 0,
 1,
 3,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 3,
 0,
 0,
 3,
 4,
 1,
 0,
 0,
 2,
 0,
 3,
 1,
 0,
 2,
 3,
 1,
 2,
 0,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 0,
 0,
 3,
 0,
 2,
 0,
 0,
 1,
 0,


In [64]:
df_dict['Actor_Rankings']=actor_rankings

In [65]:
mojotestlink=['https://www.boxofficemojo.com/title/tt2488496/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt4154796/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt0499549/?ref_=bo_se_r_1',
 'https://www.boxofficemojo.com/title/tt1825683/?ref_=bo_se_r_1']
genres=[]
for link in mojo_url_lst:
    sublist=[]
    soup=url_to_soup(link)
    genre_text=soup.find(text=re.compile('Genres'))
    try:
        first=genre_text.findNext()
        for i in first:
            clean=i.replace('\n','').strip()
            clean=clean.split()
            #print(clean)

        genres.append(clean)
    except:
        sublist.append('NA')
        genres.append(sublist)
genres

[['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Drama', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['NA'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Adventure', 'Animation', 'Comedy', 'Family', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Family', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Mystery', 'Sci-Fi', 'Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'War'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi', 'Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 'Sci-Fi', 'Thriller'],
 ['Action', 'Adventure', 'Sci-Fi'],
 ['Action', 'Adventure', 

In [66]:
ratings_clean=[]
for rating in ratings:
    if rating=='PG-13':
        rating=re.sub('PG-13','_13',rating)
        ratings_clean.append(rating)
    elif rating=='Not Rated':
        rating=re.sub('Not Rated','Unrated',rating)
        ratings_clean.append(rating)
    elif rating=='TV-MA':
        rating=re.sub('TV-MA','M',rating)
        ratings_clean.append(rating)
    elif rating=='TV-PG':
        rating=re.sub('TV-PG','P',rating)
    elif rating=='PG':
        rating=re.sub('PG','P',rating)
        ratings_clean.append(rating)
    elif rating=='TV-Y7':
        rating=re.sub('TV-Y7','G',rating)
        ratings_clean.append(rating)
    elif rating=='TV-14':
        rating=re.sub('TV-14','13',rating)
        ratings_clean.append(rating)
    elif rating=='TV-G':
        rating=re.sub('TV-G','G',rating)
        ratings_clean.append(rating)
    else:
        ratings_clean.append(rating)
#G,PG,PG13,M,R

In [67]:
for i in range(0,6):
    ratings_clean.append('NA')
len(ratings_clean)

1700

In [68]:
ratings_distinct=[]

for rating in ratings_clean:
    if rating not in ratings_distinct:
        ratings_distinct.append(rating)

In [69]:
ratings_distinct2=[]

for rating in ratings:
    if rating not in ratings_distinct2:
        ratings_distinct2.append(rating)

In [70]:
ratings_distinct

['_13', 'P', 'R', 'G', 'Approved', 'Unrated', 'M', 'Passed', 'GP', 'NA', '13']

In [71]:
ratings_list=['_13',
 'P','R',
 'G',
 'Approved',
 'Unrated','M','Passed','NA']

_13=[]
P=[]
R=[]
G=[]
Approved=[]
Unrated=[]
M=[]
Passed=[]
NA=[]


lstoflsts_r=[_13,P,R,G,Approved,Unrated,M,Passed,NA]
for rating in ratings_clean:
    for item in ratings_list:
        if item in rating:
            #print(lstoflsts[genre_list.index(item)])
            lstoflsts_r[ratings_list.index(item)].append(1)
        else:
            lstoflsts_r[ratings_list.index(item)].append(0)


In [72]:
df_dict['PG13']=_13
df_dict['PG']=P
df_dict['R']=R
df_dict['G']=G
df_dict['Approved']=Approved
df_dict['Unrated']=Unrated
df_dict['M']=M
df_dict['Passed']=Passed
df_dict['NA']=NA


In [73]:
rating_dict={}
rating_dict['PG13']=_13
rating_dict['PG']=P
rating_dict['R']=R
rating_dict['G']=G
rating_dict['Approved']=Approved
rating_dict['Unrated']=Unrated
rating_dict['M']=M
rating_dict['Passed']=Passed
rating_dict['NA']=NA

In [74]:
genres_distinct=[]
for sublist in genres:
    for genre in sublist:
        if genre not in genres_distinct:
            genres_distinct.append(genre)


In [75]:
genres_distinct

['Action',
 'Adventure',
 'Sci-Fi',
 'Drama',
 'Fantasy',
 'NA',
 'Animation',
 'Comedy',
 'Family',
 'Mystery',
 'Thriller',
 'War',
 'Crime',
 'Romance',
 'Horror',
 'Sport',
 'Western',
 'Music',
 'Musical',
 'Documentary',
 'Biography',
 'History']

In [76]:
genre_list=['Action',
 'Adventure','Drama',
 'Fantasy',
 'Animation',
 'Comedy',
 'Family','Mystery',
 'Thriller',
 'War',
 'Crime',
 'Romance',
 'Horror',
 'Western',
 'Biography']

Action=[]
Adventure=[]
Drama=[]
Fantasy=[]
Animation=[]
Comedy=[]
Family=[]
Mystery=[]
Thriller=[]
War=[]
Crime=[]
Romance=[]
Horror=[]
Western=[]
Biography=[]

lstoflsts=[Action,Adventure,Drama,Fantasy,
           Animation,Comedy,Family,Mystery,Thriller,
           War,Crime,Romance,Horror,Western,Biography]
for genre in genres:
    for item in genre_list:
        if item in genre:
            #print(lstoflsts[genre_list.index(item)])
            lstoflsts[genre_list.index(item)].append(1)
        else:
            lstoflsts[genre_list.index(item)].append(0)

In [77]:
len(Fantasy)

1700

In [78]:
imdb_url_lst

['https://www.imdb.com//title/tt2488496/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4154796/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0499549/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt1825683/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4154756/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0369610/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0848228/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt2527336/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt3606756/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt3748528/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt2527338/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0120915/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt2395427/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt0083866/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4154664/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt1951264/?ref_=adv_li_tt',
 'https://www.imdb.com//title/tt4881806/?ref_=adv_li_tt',
 'https://www.

In [28]:
imdbtestlst=['https://www.imdb.com//title/tt2488496/?ref_=adv_li_tt']
budgets=[]
budgetstest=[]
test2=['https://www.imdb.com//title/tt2488496/?ref_=adv_li_tt',
       'https://www.imdb.com/title/tt9016974/?ref_=nv_sr_srsg_0',
      'https://www.imdb.com//title/tt0316654/?ref_=adv_li_tt']
#imdb_url_lst
for link in test2:
    soup = url_to_soup(link)
    try:
        if soup.find(text=re.compile('Budget:')) != None:
            
            
            budget = soup.find(text=re.compile('Budget:'))
            print('budget: ',budget)
            #print(budget.find_next('span', class_='money').text)
            budgetstest.append(budget)

        else:
            print('no')
            budgetstest.append('second_else')

    except:
        budgetstest.append('except')

no


In [19]:
budgetstest


['except', 'second_else', 'except']

In [30]:
imdbtestlst=['https://www.imdb.com//title/tt2488496/?ref_=adv_li_tt']
budgets=[]
budgetstest=[]
test2=['https://www.imdb.com//title/tt2488496/?ref_=adv_li_tt',
       'https://www.imdb.com/title/tt9016974/?ref_=nv_sr_srsg_0',
      'https://www.imdb.com//title/tt0316654/?ref_=adv_li_tt']
#imdb_url_lst
for link in test2:
    soup = url_to_soup(link)
    try:
        if soup.find(text=re.compile('Budget:')) != None:
            if soup.find(id='titleDetails').find_all(class_='txt-block')[6]:
                content=soup.find(id='titleDetails').find_all(class_='txt-block')[6]
                content=content.get_text()
                content=re.sub('\D','',content)
                print(content)
                budgetstest.append(content)
            else:
                budgetstest.append('NA')
        else:
            budgetstest.append('NA')

    except:
        budgetstest.append('NA')
'''
    budget_text=soup.find(text=re.compile('Budget:'))
    print(budget_text)
    first=budget_text.findNext()
    '''

245000000
8815622742004


"\n    budget_text=soup.find(text=re.compile('Budget:'))\n    print(budget_text)\n    first=budget_text.findNext()\n    "

In [31]:
budgetstest

['245000000', 'NA', '8815622742004']

In [99]:
budgets[27]

'8815622742004'

In [100]:
imdb_url_lst[27]

'https://www.imdb.com//title/tt0316654/?ref_=adv_li_tt'

In [84]:
budgets[::-1]

['NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 '',
 '',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 'NA',
 '',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 '50000000',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 'NA',
 '',
 'NA',
 '',
 '',
 'NA',
 '',
 '',
 'NA',
 '',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 '',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '2351',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 '85',
 'NA',
 '',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 '',
 'NA',
 'NA',
 'NA',
 '120',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 'NA',
 '600000',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '',
 'NA',
 '

In [85]:
df_dict['Budget']=budgets

In [86]:
df_dict['Action']=Action
df_dict['Adventure']=Adventure
df_dict['Drama']=Drama
df_dict['Fantasy']=Fantasy
df_dict['Animation']=Animation
df_dict['Comedy']=Comedy
df_dict['Family']=Family
df_dict['Mystery']=Mystery
df_dict['Thriller']=Thriller
df_dict['War']=War
df_dict['Crime']=Crime
df_dict['Romance']=Romance
df_dict['Horror']=Horror
df_dict['Western']=Western
df_dict['Biography']=Biography

In [87]:
occurrences=Action.count(1)

In [88]:
occurrences

640

In [89]:
df_dict.keys()

dict_keys(['Title', 'Revenue', 'Runtime', 'Actor_Rankings', 'PG13', 'PG', 'R', 'G', 'Approved', 'Unrated', 'M', 'Passed', 'NA', 'Budget', 'Action', 'Adventure', 'Drama', 'Fantasy', 'Animation', 'Comedy', 'Family', 'Mystery', 'Thriller', 'War', 'Crime', 'Romance', 'Horror', 'Western', 'Biography'])

In [90]:
movies_df=pd.DataFrame.from_dict(df_dict)

In [91]:
movies_df.head()

,Title,Revenue,Runtime,Actor_Rankings,PG13,PG,R,G,Approved,Unrated,...,Comedy,Family,Mystery,Thriller,War,Crime,Romance,Horror,Western,Biography
0,Star Wars: Episode VII - The Force Awakens,2068455677,138,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Avengers: Endgame,2797501328,181,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Avatar,2847246203,162,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Black Panther,1347597973,134,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Avengers: Infinity War,2048359754,149,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
print(movies_df.columns.tolist())

['Title', 'Revenue', 'Runtime', 'Actor_Rankings', 'PG13', 'PG', 'R', 'G', 'Approved', 'Unrated', 'M', 'Passed', 'NA', 'Budget', 'Action', 'Adventure', 'Drama', 'Fantasy', 'Animation', 'Comedy', 'Family', 'Mystery', 'Thriller', 'War', 'Crime', 'Romance', 'Horror', 'Western', 'Biography']


In [93]:
#movies_df.to_csv('movies_lr_Final_fixedbudgets.csv')

In [94]:
ratings_df=pd.DataFrame.from_dict(rating_dict)

In [95]:
ratings_df.to_csv('ratings.csv')

In [96]:
len(ratings)

1700

In [97]:
budgets[27]

'8815622742004'